# Lab 15: Microservices Integration**Duration:** 45 minutes  **Objective:** Integrate Redis with microservices architecture## 🎯 Learning Objectives- Implement service discovery- Build event-driven communication- Create distributed caching- Implement saga patterns- Handle distributed transactions

In [ ]:
import redisimport jsonimport uuidimport timefrom datetime import datetimefrom colorama import init, Fore, Styleinit(autoreset=True)r = redis.Redis(host='localhost', port=6379, decode_responses=True)print(f"{Fore.GREEN}✅ Lab 15: Microservices Integration{Style.RESET_ALL}")

## Microservices Patterns Implementation

In [ ]:
class ServiceRegistry:    """Service discovery and registration."""        def __init__(self, redis_client):        self.r = redis_client        self.ttl = 30        def register_service(self, name, host, port, metadata=None):        service_id = f"{name}:{host}:{port}"                service_data = {            'id': service_id,            'name': name,            'host': host,            'port': port,            'metadata': json.dumps(metadata or {}),            'registered_at': datetime.now().isoformat(),            'health': 'healthy'        }                # Store service data        service_key = f"service:{service_id}"        self.r.hset(service_key, mapping=service_data)        self.r.expire(service_key, self.ttl)                # Add to service index        self.r.sadd(f"services:{name}", service_id)                return service_id        def discover_service(self, name):        service_ids = self.r.smembers(f"services:{name}")        services = []                for service_id in service_ids:            service_key = f"service:{service_id}"            if self.r.exists(service_key):                service_data = self.r.hgetall(service_key)                services.append(service_data)            else:                self.r.srem(f"services:{name}", service_id)                return servicesclass EventBus:    """Event-driven communication between services."""        def __init__(self, redis_client):        self.r = redis_client        def publish_event(self, event_type, service, data):        event = {            'id': str(uuid.uuid4()),            'type': event_type,            'service': service,            'timestamp': datetime.now().isoformat(),            'data': data        }                # Publish to channel        channel = f"events:{event_type}"        self.r.publish(channel, json.dumps(event))                # Store in stream for persistence        stream_key = f"event_stream:{event_type}"        self.r.xadd(stream_key, event)                return event['id']        def subscribe_events(self, event_types):        pubsub = self.r.pubsub()                for event_type in event_types:            channel = f"events:{event_type}"            pubsub.subscribe(channel)                return pubsubclass DistributedCache:    """Distributed caching for microservices."""        def __init__(self, redis_client):        self.r = redis_client        def cache_for_service(self, service, key, value, ttl=300):        cache_key = f"cache:{service}:{key}"        self.r.setex(cache_key, ttl, json.dumps(value))        def get_from_cache(self, service, key):        cache_key = f"cache:{service}:{key}"        value = self.r.get(cache_key)        return json.loads(value) if value else None        def invalidate_service_cache(self, service):        pattern = f"cache:{service}:*"        keys = self.r.keys(pattern)        if keys:            self.r.delete(*keys)        return len(keys)# Test microservices integrationprint(f"{Fore.CYAN}=== SERVICE REGISTRY ==={Style.RESET_ALL}\n")registry = ServiceRegistry(r)# Register servicesservices = [    ('api-gateway', 'gateway.local', 8080),    ('user-service', 'users.local', 8081),    ('order-service', 'orders.local', 8082),    ('payment-service', 'payments.local', 8083)]for name, host, port in services:    service_id = registry.register_service(name, host, port, {'version': '1.0'})    print(f"Registered: {service_id}")# Discover servicesdiscovered = registry.discover_service('user-service')print(f"\nDiscovered user-service instances: {len(discovered)}")# Event busprint(f"\n{Fore.CYAN}=== EVENT BUS ==={Style.RESET_ALL}\n")event_bus = EventBus(r)# Publish eventsevents = [    ('user.created', 'user-service', {'user_id': 'USR-123', 'email': 'user@example.com'}),    ('order.placed', 'order-service', {'order_id': 'ORD-456', 'total': 99.99}),    ('payment.processed', 'payment-service', {'payment_id': 'PAY-789', 'status': 'success'})]for event_type, service, data in events:    event_id = event_bus.publish_event(event_type, service, data)    print(f"Published {event_type}: {event_id}")# Distributed cacheprint(f"\n{Fore.CYAN}=== DISTRIBUTED CACHE ==={Style.RESET_ALL}\n")cache = DistributedCache(r)# Cache data for servicescache.cache_for_service('user-service', 'user:123', {'name': 'John', 'role': 'admin'})cache.cache_for_service('order-service', 'order:456', {'status': 'shipped', 'items': 3})# Retrieve from cacheuser_data = cache.get_from_cache('user-service', 'user:123')print(f"Cached user data: {user_data}")order_data = cache.get_from_cache('order-service', 'order:456')print(f"Cached order data: {order_data}")print("\n✅ Microservices integration completed")